In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Set fileName, and run all.
# logs.csv will contain the required data

fileName = 'btcusdt_1h.csv'

In [ ]:
df = pd.read_csv(fileName)

In [ ]:
def ESMA(df):
  length = 30
  multiplier = 3
  atrLength = 14
  exp = True
  def ESMA(df,exp, period, column="close"):
    if exp == True:
      return df[column].ewm(span=period, adjust=False).mean()
    else:
      return df[column].rolling(window=period).mean()

  df["ma"] = ESMA(df,exp,length)
  # rangema = ATR LOGIC #### TODO ####
  df['High-Low'] = df['high'] - df['low']
  df['High-PrevClose'] = np.abs(df['high'] - df['close'].shift(1))
  df['Low-PrevClose'] = np.abs(df['low'] - df['close'].shift(1))
  df['TrueRange'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
  df["rangema"] = df['TrueRange'].rolling(window=atrLength).mean()
  df["upper"] = df["ma"] + df["rangema"]*multiplier
  df["lower"] = df["ma"] - df["rangema"] *multiplier
  df["crossUpper"] = 0
  df["crossLower"] = 0
  df["bprice"] = 0
  df["sprice"] = 0
  df["BCond"] = 0
  df["SCond"] = 0  # Same as cross upper and cross lower(unless there are nan values, none in our case)
  df["CancelBCond"] = 0
  df["CancelSCond"] = 0
  for i in range(1,len(df)):
    df.loc[i,"crossUpper"] = int(df.loc[i-1, "close"]< df.loc[i-1, "upper"] and df.loc[i,"close"]>df.loc[i,"upper"])
    df.loc[i,"crossLower"] = int(df.loc[i-1, "close"] > df.loc[i-1, "lower"] and df.loc[i,"close"]<df.loc[i,"lower"])
    df.loc[i,"bprice"] = df.loc[i,"high"] + df.loc[:i,"high"].min() if df.loc[i,"crossUpper"] == 1 else df.loc[i-1,"bprice"]
    df.loc[i,"sprice"] = df.loc[i,"low"] - df.loc[:i,"low"].min() if df.loc[i,"crossLower"] == 1 else df.loc[i-1,"bprice"]
    df.loc[i,"BCond"] = df.loc[i,"close"]>df.loc[i,"upper"]
    df.loc[i,"SCond"] = df.loc[i,"close"]<df.loc[i,"lower"]
    df.loc[i,"CancelBCond"] = df.loc[i,"BCond"] and (df.loc[i,"close"]<df.loc[i,"ma"] or df.loc[i,"high"]>=df.loc[i,"bprice"])
    df.loc[i, "CancelSCond"] = df.loc[i,"SCond"] and (df.loc[i,"close"]>df.loc[i,"ma"] or df.loc[i,"low"]<=df.loc[i,"sprice"])
  position = 0

  df["signals"] = 0
  for i in range(20,len(df)):
    if df.loc[i,"BCond"] == 1:
      if position != 1:
        df.loc[i,"signals"] = 1
        position +=1
    elif df.loc[i,"SCond"]==1:
      if position != -1:
        df.loc[i,"signals"] = -1
        position -= 1
    elif df.loc[i,"CancelBCond"] == 1 and position == 1:
      df.loc[i,"signals"] = -1
      position -=1
    elif df.loc[i,"CancelSCond"] == 1 and position == -1:
      df.loc[i,"signals"] = 1
      position +=1
    else:
      df.loc[i,"signals"] = 0

  df['Index'] = range(0,len(df))
  df = df[['Index','datetime','open','high','low','close','volume','signals']]
  df.to_csv('logs.csv',encoding='utf-8',index=False)

In [ ]:
ESMA(df)